# Capstone Project - Data Preprocessing

**Hazırlayan:** Zeynep Sarı

In [2]:
# import dependencies
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
import glob
import os
import ast

## Data Prep

In [3]:
# Load the Data
folder_path = f"{os.getcwd()}/ttdata"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
dfs = []

for csv in csv_files:
    dfs.append(pd.read_csv(csv))
df = pd.concat(dfs)
del dfs # Delete to save some memo
df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,apps,churn
0,6549e0e5-8e85-4037-b52e-31dd3dec6624,23,45.0,Prepaid,14.27,146.62,4.70,1761.84,0,NaN,74,13.0,14,5.76,[],False
1,e7091181-b875-4e74-bc34-992cd37e7dee,32,15.0,Broadband,NaN,104.61,NaN,366.11,1,0.0,0,NaN,13,7.67,[],False
2,26e3bdf7-0b20-4790-b7a8-5b626b80033a,33,16.0,Postpaid,31.66,28.50,19.75,582.74,4,1.0,0,20.0,20,3.59,"['İzleGo', 'Konuşalım']",False
3,78f92350-7189-44be-8304-3ece87c88de7,19,14.0,Broadband,NaN,NaN,NaN,425.17,1,0.0,0,NaN,20,9.71,['RitimGo'],False
4,95aa23c8-68ec-4c33-8d10-5ed0678ff4c6,44,249.0,Postpaid,77.84,NaN,15.15,843.20,5,0.0,0,17.0,8,9.34,[],False


In [4]:
# Check empty cols
df.isnull().sum()/len(df)*100

id                         0.00000
age                        0.00000
tenure                     0.24800
service_type               0.00000
avg_call_duration         36.64313
data_usage                 4.99533
roaming_usage             33.31212
monthly_charge             2.50071
overdue_payments           0.00000
auto_payment              33.44748
avg_top_up_count           0.00000
call_drops                33.31212
customer_support_calls     0.00000
satisfaction_score         0.00000
apps                       0.00000
churn                      0.00000
dtype: float64

In [5]:
# Set the cols
fill_cols = df.columns[list(df.isnull().sum()/len(df)*100 > 0)].to_list() # Columns to be filled

# Fill columns with its average
df[fill_cols] = df[fill_cols].apply(lambda col: col.fillna(col.mean()))

# Check for Nulls
df.isnull().sum()/len(df)*100

id                        0.0
age                       0.0
tenure                    0.0
service_type              0.0
avg_call_duration         0.0
data_usage                0.0
roaming_usage             0.0
monthly_charge            0.0
overdue_payments          0.0
auto_payment              0.0
avg_top_up_count          0.0
call_drops                0.0
customer_support_calls    0.0
satisfaction_score        0.0
apps                      0.0
churn                     0.0
dtype: float64

In [6]:
# One Hot Encode apps column, using MLB since apps columns dtype is list

# OHE apps column 
df['apps'] = df['apps'].apply(ast.literal_eval) # Convert string to list

mlb = MultiLabelBinarizer()

encoded = mlb.fit_transform(df['apps'])

meta_df = pd.DataFrame(encoded, columns=mlb.classes_)

ohe_df = df.drop(columns=['apps']).join(meta_df)

# OHE service_type column
final_df = pd.get_dummies(ohe_df, columns=['service_type'])

In [7]:
final_df.describe()

,age,tenure,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,CüzdanX,HızlıPazar,Konuşalım,RitimGo,İzleGo
count,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07,1.000000e+07
mean,3.037627e+01,1.582518e+02,6.051176e+01,1.000657e+02,2.999117e+01,7.437683e+02,1.666214e+00,4.998284e-01,1.752045e+01,1.000296e+01,1.000299e+01,5.498994e+00,1.010300e-02,9.937000e-03,2.002380e-01,1.000090e-01,2.973200e-02
std,1.140044e+01,9.463726e+01,2.733607e+01,5.624355e+01,1.414437e+01,5.897940e+02,1.825839e+00,4.078986e-01,2.948364e+01,4.945454e+00,6.055363e+00,2.598768e+00,1.000047e-01,9.918799e-02,4.001784e-01,3.000120e-01,1.698470e-01
min,1.800000e+01,1.000000e+00,1.000000e+00,1.000000e-01,0.000000e+00,3.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+01,7.800000e+01,4.795000e+01,5.273000e+01,2.248000e+01,3.219500e+02,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,5.000000e+00,3.250000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.800000e+01,1.560000e+02,6.051176e+01,1.000657e+02,2.999117e+01,4.889600e+02,1.000000e+00,4.998284e-01,0.000000e+00,1.000296e+01,1.000000e+01,5.500000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.800000e+01,2.340000e+02,7.306000e+01,1.474200e+02,3.749000e+01,1.003240e+03,3.000000e+00,1.000000e+00,2.900000e+01,1.300000e+01,1.500000e+01,7.750000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,8.000000e+01,7.540000e+02,1.200000e+02,2.000000e+02,6.000000e+01,2.500000e+03,5.000000e+00,1.000000e+00,1.000000e+02,2.000000e+01,2.000000e+01,1.000000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000000 entries, 0 to 999999
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      object 
 1   age                     int64  
 2   tenure                  float64
 3   avg_call_duration       float64
 4   data_usage              float64
 5   roaming_usage           float64
 6   monthly_charge          float64
 7   overdue_payments        int64  
 8   auto_payment            float64
 9   avg_top_up_count        int64  
 10  call_drops              float64
 11  customer_support_calls  int64  
 12  satisfaction_score      float64
 13  churn                   bool   
 14  CüzdanX                 int64  
 15  HızlıPazar              int64  
 16  Konuşalım               int64  
 17  RitimGo                 int64  
 18  İzleGo                  int64  
 19  service_type_Broadband  bool   
 20  service_type_Postpaid   bool   
 21  service_type_Prepaid    bool   
dtyp

In [9]:
# Convert all boolean columns to int
final_df[final_df.select_dtypes(bool).columns] = final_df.select_dtypes(bool).astype(int)
# Set the index col
final_df.set_index("id", inplace=True)
final_df.head()

,age,tenure,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,...,satisfaction_score,churn,CüzdanX,HızlıPazar,Konuşalım,RitimGo,İzleGo,service_type_Broadband,service_type_Postpaid,service_type_Prepaid
id,,,,,,,,,,,,,,,,,,,,,
6549e0e5-8e85-4037-b52e-31dd3dec6624,23,45.0,14.270000,146.620000,4.700000,1761.84,0,0.499828,74,13.000000,...,5.76,0,0,0,0,0,0,0,0,1
e7091181-b875-4e74-bc34-992cd37e7dee,32,15.0,60.511764,104.610000,29.991167,366.11,1,0.000000,0,10.002964,...,7.67,0,0,0,0,0,0,1,0,0
26e3bdf7-0b20-4790-b7a8-5b626b80033a,33,16.0,31.660000,28.500000,19.750000,582.74,4,1.000000,0,20.000000,...,3.59,0,0,0,1,0,1,0,1,0
78f92350-7189-44be-8304-3ece87c88de7,19,14.0,60.511764,100.065695,29.991167,425.17,1,0.000000,0,10.002964,...,9.71,0,0,0,0,1,0,1,0,0
95aa23c8-68ec-4c33-8d10-5ed0678ff4c6,44,249.0,77.840000,100.065695,15.150000,843.20,5,0.000000,0,17.000000,...,9.34,0,0,0,0,0,0,0,1,0


In [10]:
final_df.to_parquet("preprocessed_data.parquet")